In [1]:
import lore
from datamanager import *

from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [18]:
## Iris Dataset
# dataset_name = 'dataset/iris.csv'
# dataset = prepare_iris_dataset(dataset_name)

## german
dataset_name = 'dataset/german_credit.csv'
dataset = prepare_german_dataset(dataset_name)

## adult
# dataset_name = 'dataset/adult.csv'
# dataset = prepare_adult_dataset(dataset_name)

## compas-scores-two-years
# dataset_name = 'dataset/compas-scores-two-years.csv'
# dataset = prepare_compass_dataset(dataset_name)

## wine
# dataset_name = 'dataset/wine.csv'
# dataset = prepare_wine_dataset(dataset_name)

dataframe = dataset[0]
class_name = dataset[1]
dataset_fin = prepare_dataset(dataframe, class_name)

In [19]:
dataframe.head()

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,...,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,...,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,...,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,...,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes


In [20]:
df = dataset_fin[0]
feature_names = dataset_fin[1]
class_values = dataset_fin[2]
numeric_columns = dataset_fin[3]
rdf = dataset_fin[4]
real_feature_names = dataset_fin[5]
features_map = dataset_fin[6]

X = df.loc[:, df.columns != class_name].values
y = df[class_name].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
blackbox = RandomForestClassifier()
blackbox.fit(X_train, y_train)

RandomForestClassifier()

In [21]:
y_pred = blackbox.predict(X_test)
print('Accuracy %.3f' % accuracy_score(y_test, y_pred))

Accuracy 0.773


In [22]:
class_values

[0, 1]

In [23]:
i = 10
x = X_test[i]
y_val = blackbox.predict(x.reshape(1,-1))[0]

y_val_name = class_values[y_val]
print(y_val_name)

0


In [24]:

lore_obj = lore.LORE(X_test, blackbox, feature_names, class_name, class_values,
                 numeric_columns, features_map, verbose=True)

Z = lore_obj.neighgen_fn(x)

print('Z is:',Z)

explanation = lore_obj.explain_instance(x, y_val, blackbox, nbr_runs=10, verbose=True)


Z is: [[9.00000000e+00 1.23600000e+03 1.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [8.94002889e+00 1.23598213e+03 9.62213392e-01 ... 9.82274937e-01
  7.98748545e-03 9.48329238e-01]
 [9.00229202e+00 1.23594906e+03 1.06069613e+00 ... 9.95798381e-01
  2.06936721e-02 1.02355754e+00]
 ...
 [8.17271848e+00 1.19941114e+03 1.03355598e+00 ... 1.15814403e+00
  5.57965917e-02 1.03869497e+00]
 [8.16325165e+00 1.19942790e+03 9.87842548e-01 ... 1.12991193e+00
  6.21252708e-02 1.01161168e+00]
 [8.12777857e+00 1.19947932e+03 1.00328635e+00 ... 1.16416183e+00
  2.74947380e-02 1.05783091e+00]]
generating neighborhood - ngmusx
synthetic neighborhood class counts {0: 255, 1: 755}
learning local decision tree
retrieving explanation


In [25]:
print(explanation)

r = { credit_history != all credits at this bank paid back duly, telephone != none } --> { default: 0 }
c = { { telephone = none, account_check_status != no checking account } }
